# 🔧 Model Optimization (Hyperparameter Tuning)
XGBoost için en iyi hiperparametreleri buluyoruz.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [ ]:
train = pd.read_csv('../data/train.csv')
stores = pd.read_csv('../data/stores.csv')
features = pd.read_csv('../data/features.csv')
train['Date'] = pd.to_datetime(train['Date'])
features['Date'] = pd.to_datetime(features['Date'])

In [ ]:
data = train.merge(stores, on='Store', how='left')
data = data.merge(features, on=['Store','Date','IsHoliday'], how='left')
data.fillna(0, inplace=True)

In [ ]:
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Week'] = data['Date'].dt.isocalendar().week.astype(int)
le = LabelEncoder()
data['Type'] = le.fit_transform(data['Type'])

In [ ]:
y = data['Weekly_Sales']
X = data.drop(['Weekly_Sales','Date'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 🎯 Randomized Search CV

In [ ]:
params = {
    'n_estimators': [200, 300, 500],
    'max_depth': [6, 10, 15],
    'learning_rate': [0.05, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}
model = XGBRegressor(random_state=42)
search = RandomizedSearchCV(model, params, n_iter=5, cv=3, scoring='neg_mean_squared_error', verbose=1)
search.fit(X_train, y_train)
best_model = search.best_estimator_
best_model

## 🧪 Sonuç Değerlendirmesi

In [ ]:
y_pred = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('Optimized RMSE:', rmse)